In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
import numpy as np
import matplotlib.pyplot as plt
import os, sys, shutil, copy, time, random

import sys
sys.path.append('../../')

In [3]:
from torchuq.metric.interval import *
from torchuq.metric import distribution, quantile, interval
from torchuq.transform.naive import *
from torchuq.transform.conformal import *
from torchuq.transform.basic import *

torchuq.transform.basic


In [4]:
n_pred = 200
def generate_predictions(n_pred):
    pred_distribution = Normal(loc=torch.linspace(0, 2, n_pred)[torch.randperm(n_pred)], scale=torch.ones(n_pred))
    labels = pred_distribution.sample() 

    pred_point = distribution_to_point(pred_distribution, functional='mean') 
    pred_interval = distribution_to_interval(pred_distribution, confidence=0.9)
    pred_quantile = distribution_to_quantile(pred_distribution, n_quantiles=10)
    pred_quantile2 = distribution_to_quantile(pred_distribution, quantiles=torch.linspace(0, 1, 12)[1:-1].pow(3))   # Get a strange quantile choice to make sure things still work
    pred_particle = distribution_to_particle(pred_distribution, n_particles=30)
    pred_ensemble = {'point_1': pred_point, 'interval_1': pred_interval, 'quantile_1': pred_quantile} 
    
    prediction_list = [pred_particle, pred_distribution, pred_point, pred_interval, pred_quantile, pred_quantile2, pred_ensemble]
    prediction_type = ['particle', 'distribution', 'point', 'interval', 'quantile', 'quantile', 'ensemble']

    return prediction_list, prediction_type, labels

prediction_list, prediction_type, labels = generate_predictions(n_pred)

In [ ]:
# Test to check that score and iscore are indeed inverse of each other
from torchuq.transform.conformal import _conformal_iscore_functions, _conformal_score_functions
values = torch.linspace(1, 2, 100).view(-1, 1)

for prediction, ptype in zip(prediction_list, prediction_type):
    for i in range(2):
        score_name = '%s_%d' % (ptype, i) 
        if score_name in _conformal_score_functions.keys():
            reconstruction = _conformal_iscore_functions[score_name](prediction, _conformal_score_functions[score_name](prediction, values))
            print(score_name, reconstruction.shape, (reconstruction - values).abs().sum().item())  # The shape should be [100, 200] and all the reconstruction error should be 0
    
# Test to check that iscore and score are indeed inverse of each other (other composition direction)
values = torch.linspace(0, 1, 100).view(-1, 1)
for prediction, ptype in zip(prediction_list, prediction_type):
    for i in range(2):
        score_name = '%s_%d' % (ptype, i) 
        if score_name in _conformal_score_functions.keys():
            reconstruction = _conformal_score_functions[score_name](prediction, _conformal_iscore_functions[score_name](prediction, values))
            print(score_name, reconstruction.shape, (reconstruction - values).abs().sum().item())  # The shape should be [100, 200] and all the reconstruction error should be 0
            
# Note that it is normal for the interval_1 to have large error

In [4]:
# Step by step testing

from torchuq.transform.conformal import _conformal_score_functions
from torchuq.transform.conformal import _conformal_iscore_functions

# pred_point[10:20] = pred_point[9]
# labels[10:20] = labels[9]

pred_point = torch.Tensor([1., 2., 3., 3., 3., 4., 4., 4., 6., 6.])
labels = torch.zeros(len(pred_point))

predictions = torch.Tensor([1., 2., 3., 3., 3.])

score_func = _conformal_score_functions['point_0']
iscore_func = _conformal_iscore_functions['point_0']
val_scores = torch.sort(score_func(pred_point, labels).abs().flatten())[0]
val_scores_ge = torch.linspace(0, 1, len(val_scores)+2)[:-1]  # Generate 0, 1/N+1, ..., N/N+1
val_scores_geq = torch.linspace(0, 1, len(val_scores)+2)[:-1]
print(len(val_scores))

# Compute the quantiles of the non-conformity scores, and handle situations where the quantiles are identical. 
while True:
    new_val_scores_ge = val_scores_ge.clone()
    new_val_scores_ge[1:-1][val_scores[:-1] == val_scores[1:]] = val_scores_ge[2:][val_scores[:-1] == val_scores[1:]]
    if (new_val_scores_ge - val_scores_ge).sum() == 0:
        break
    val_scores_ge = new_val_scores_ge

val_scores_geq = val_scores_ge[:-1] + (val_scores_ge[1:] - val_scores_ge[:-1]).view(1, -1) * torch.rand(len(predictions)).view(-1, 1)
val_scores_ge = val_scores_ge[1:].view(1, -1).repeat(len(predictions), 1)

while True:
    new_val_scores_geq = val_scores_geq.clone()
    new_val_scores_geq[:, 1:][:, val_scores[:-1] == val_scores[1:]] = val_scores_geq[:, :-1][:, val_scores[:-1] == val_scores[1:]]
    if (new_val_scores_geq - val_scores_geq).sum() == 0:
        break
    val_scores_geq = new_val_scores_geq


# val_scores_ge[i] is 1/(N+1)#{n | A(x_n, y_n) < a}, if a > val_scores[i]
# val_scores_geq[i] is 1/(N+1)#{n | A(x_n, y_n) < a} + U(0, 1)/(N+1)#{n | A(x_n, y_n) = a} if a = val_scores[i]

confidence = 0.95
coverage = 'exact'

if coverage == 'exact':   # If coverage is exact, add U(0, 1)/(N+1)
    noise = torch.rand(len(predictions), 1) / (len(val_scores) + 1)
    val_scores_geq = val_scores_geq + noise
    val_scores_ge = val_scores_ge + noise
else:   # If coverage is up to 1/N accuracy, then add 1/(N+1)
    val_scores_geq = val_scores_geq + 1/(len(val_scores) + 1)
    val_scores_ge = val_scores_ge + 1/(len(val_scores) + 1)
    
torch.set_printoptions(precision=3)
print(val_scores_ge)
print(val_scores_geq)

geq_index = (val_scores_geq <= confidence).type(torch.int32).sum(dim=1) 
ge_index = (val_scores_ge <= confidence).type(torch.int32).sum(dim=1) 

eps = 1e-5 
torch.set_printoptions(precision=5)
target_scores = torch.maximum((val_scores[ge_index.clamp(max=len(val_scores)-1)] - eps), val_scores[geq_index-1])
y_ub = iscore_func(predictions, target_scores.view(1, -1)).flatten()
y_lb = iscore_func(predictions, -target_scores.view(1, -1)).flatten()
y_ub[ge_index > len(val_scores)-1] = float('inf')
y_lb[ge_index > len(val_scores)-1] = -float('inf')

result = torch.stack([y_lb, y_ub], axis=-1)

print(ge_index, geq_index)

print(target_scores)
print(result)

10
tensor([[0.134, 0.225, 0.497, 0.497, 0.497, 0.770, 0.770, 0.770, 0.952, 0.952],
        [0.126, 0.217, 0.490, 0.490, 0.490, 0.762, 0.762, 0.762, 0.944, 0.944],
        [0.173, 0.264, 0.537, 0.537, 0.537, 0.810, 0.810, 0.810, 0.991, 0.991],
        [0.092, 0.183, 0.456, 0.456, 0.456, 0.728, 0.728, 0.728, 0.910, 0.910],
        [0.162, 0.252, 0.525, 0.525, 0.525, 0.798, 0.798, 0.798, 0.980, 0.980]])
tensor([[0.068, 0.159, 0.300, 0.300, 0.300, 0.573, 0.573, 0.573, 0.820, 0.820],
        [0.092, 0.183, 0.388, 0.388, 0.388, 0.661, 0.661, 0.661, 0.877, 0.877],
        [0.132, 0.223, 0.414, 0.414, 0.414, 0.687, 0.687, 0.687, 0.909, 0.909],
        [0.023, 0.113, 0.247, 0.247, 0.247, 0.520, 0.520, 0.520, 0.771, 0.771],
        [0.107, 0.198, 0.362, 0.362, 0.362, 0.635, 0.635, 0.635, 0.871, 0.871]])
tensor([ 8, 10,  8, 10,  8]) tensor([10, 10, 10, 10, 10])
tensor([6., 6., 6., 6., 6.])
tensor([[-5.,  7.],
        [-inf, inf],
        [-3.,  9.],
        [-inf, inf],
        [-3.,  9.]])


In [48]:
n_pred = 100

from torchuq.transform.conformal import _conformal_score_functions, _conformal_iscore_functions, _concat_predictions
from torchuq import _get_prediction_batch_shape
class ConformalIntervalPredictor(ConformalBase):
    def __init__(self, input_type='interval', coverage='exact', score_func=0, confidence=0.95, verbose=False):
        """
        Inputs:
            input_type: str, one of the regression input types
            score_func: int, the score function to use. The index corresponds to the paper (cite). 
            coverage: the coverage can be 'exact' or '1/n'. If the coverage is exact, then the algorithm can output [-inf, +inf] intervals
        """
        super(ConformalIntervalPredictor, self).__init__(input_type=input_type, score_func=score_func, verbose=verbose)
        self.val_scores = None
        
        assert coverage == 'exact' or coverage == '1/n' or coverage == '1/N', "Coverage can only be 'exact' or '1/N'"
        self.coverage = coverage
        self.confidence = confidence
        
    
    def __call__(self, predictions, confidence=None):
        """
        Input: 
            confidence: float, the confidence level of the prediction intervals. If None then uses the default confidence interval specified in the constructor 
        """
        if confidence is None:
            confidence = self.confidence 
        assert confidence > 0 and confidence < 1., 'Confidence must be a number of (0, 1)'
        
        self.check_type(predictions) 
        self.to(predictions)
        
        score_func_name = '%s_%d' % (self.input_type, self.score_func)
        score_func = _conformal_score_functions[score_func_name]
        iscore_func = _conformal_iscore_functions[score_func_name]
        
        # Get the sorted non-conformity score on the validation set 
        test_shape = _get_prediction_batch_shape(predictions)
        
        val_scores = torch.sort(score_func(_concat_predictions[self.input_type](self.predictions), torch.cat(self.labels, dim=0)).abs().flatten())[0]
        val_scores_ge = torch.linspace(0, 1, len(val_scores)+2, device=self.device)[:-1]  # Generate 0, 1/N+1, ..., N/N+1
        val_scores_geq = torch.linspace(0, 1, len(val_scores)+2, device=self.device)[:-1]
        
        # Compute the quantiles of the non-conformity scores, and handle situations where the quantiles are identical. 
        while True:   # This iteration is for handling values with identical non-conformity score 
            new_val_scores_ge = val_scores_ge.clone()
            new_val_scores_ge[1:-1][val_scores[:-1] == val_scores[1:]] = val_scores_ge[2:][val_scores[:-1] == val_scores[1:]]
            if (new_val_scores_ge - val_scores_ge).sum() == 0:
                break
            val_scores_ge = new_val_scores_ge

        val_scores_geq = val_scores_ge[:-1] + (val_scores_ge[1:] - val_scores_ge[:-1]).view(1, -1) * torch.rand(test_shape, device=self.device).view(-1, 1)
        val_scores_ge = val_scores_ge[1:].view(1, -1).repeat(test_shape, 1)

        while True:   # This iteration is for handling values with identical non-conformity score 
            new_val_scores_geq = val_scores_geq.clone()
            new_val_scores_geq[:, 1:][:, val_scores[:-1] == val_scores[1:]] = val_scores_geq[:, :-1][:, val_scores[:-1] == val_scores[1:]]
            if (new_val_scores_geq - val_scores_geq).sum() == 0:
                break
            val_scores_geq = new_val_scores_geq
        

        # Now we should have the following:
        # val_scores_ge[i] is 1/(N+1)#{n | A(x_n, y_n) < a}, if a > val_scores[i]
        # val_scores_geq[i] is 1/(N+1)#{n | A(x_n, y_n) < a} + U(0, 1)/(N+1)#{n | A(x_n, y_n) = a} if a = val_scores[i]
        
        if self.coverage == 'exact':   # If coverage is exact, add U(0, 1)/(N+1)
            noise = torch.rand(test_shape, 1, device=self.device) / (len(val_scores) + 1)
            val_scores_geq = val_scores_geq + noise
            val_scores_ge = val_scores_ge + noise
        else:   # If coverage is up to 1/N accuracy, then add 1/(N+1), we always under cover to achieve the smallest intervals 
            val_scores_geq = val_scores_geq + 1/(len(val_scores) + 1)
            val_scores_ge = val_scores_ge + 1/(len(val_scores) + 1)
        print(val_scores_ge)
        print(val_scores_geq)
        
        geq_index = (val_scores_geq <= confidence).type(torch.int32).sum(dim=1) 
        ge_index = (val_scores_ge <= confidence).type(torch.int32).sum(dim=1)     # The score of which validation sample to use
        
        eps = 1e-5 
        target_scores = torch.maximum((val_scores[ge_index.clamp(max=len(val_scores)-1)] - eps), val_scores[geq_index-1])
        print(geq_index)
        y_ub = iscore_func(predictions, target_scores.view(1, -1)).flatten()
        y_lb = iscore_func(predictions, -target_scores.view(1, -1)).flatten()
        
        y_ub[(ge_index > len(val_scores)-1) | (geq_index > len(val_scores)-1)] = float('inf')
        y_lb[(ge_index > len(val_scores)-1) | (geq_index > len(val_scores)-1)] = -float('inf')

        result = torch.stack([y_lb, y_ub], axis=-1)
        return result
    
    
calibrator = ConformalIntervalPredictor(input_type='point', coverage='1/N', confidence=0.2)
predictions = torch.linspace(0, 1, n_pred)
labels = predictions + torch.rand(n_pred) - 0.5 # torch.linspace(-1, 0.5, n_pred).clamp(min=0.0) + torch.linspace(-0.5, 1, n_pred).clamp(max=0.0)


# predictions = torch.linspace(0, 1, n_pred)
# labels = predictions.clone()

perm = torch.randperm(n_pred)
predictions = predictions[perm]
labels = labels[perm]

calibrator.train(predictions[:50], labels[:50])
pred = calibrator(predictions[50:])

print(pred)

tensor([[0.0392, 0.0588, 0.0784,  ..., 0.9608, 0.9804, 1.0000],
        [0.0392, 0.0588, 0.0784,  ..., 0.9608, 0.9804, 1.0000],
        [0.0392, 0.0588, 0.0784,  ..., 0.9608, 0.9804, 1.0000],
        ...,
        [0.0392, 0.0588, 0.0784,  ..., 0.9608, 0.9804, 1.0000],
        [0.0392, 0.0588, 0.0784,  ..., 0.9608, 0.9804, 1.0000],
        [0.0392, 0.0588, 0.0784,  ..., 0.9608, 0.9804, 1.0000]])
tensor([[0.0363, 0.0559, 0.0755,  ..., 0.9579, 0.9775, 0.9971],
        [0.0319, 0.0515, 0.0711,  ..., 0.9534, 0.9730, 0.9927],
        [0.0360, 0.0556, 0.0752,  ..., 0.9576, 0.9772, 0.9968],
        ...,
        [0.0318, 0.0514, 0.0710,  ..., 0.9534, 0.9730, 0.9926],
        [0.0355, 0.0551, 0.0747,  ..., 0.9570, 0.9766, 0.9963],
        [0.0349, 0.0545, 0.0741,  ..., 0.9565, 0.9761, 0.9957]])
tensor([ 9,  9,  9,  9, 10,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10,  9, 10,  9,
        10,  9,  9,  9,  9,  9,  9, 10,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9, 10,  9,  9,  9, 10,  9, 10, 